In [102]:
from pyspark.sql import SparkSession

from pyspark.sql import functions as F
from pyspark.sql.functions import isnan, when, count, col, udf
from pyspark.sql.types import IntegerType
from pyspark.sql.window import Window

import sys

spark = SparkSession.builder.appName("Homicide Report").getOrCreate()
sc = spark.sparkContext

In [123]:
# https://www.kaggle.com/murderaccountability/homicide-reports/data
df = spark.read.csv("./database.csv", header=True)

In [124]:
from functools import reduce
columnsRenamed = [ (c, c.replace(" ", "")) for c in df.columns]
df = reduce(lambda df, c: df.withColumnRenamed(c[0], c[1]), columnsRenamed, df)

In [105]:
df.printSchema()
df.head()

root
 |-- RecordID: string (nullable = true)
 |-- AgencyCode: string (nullable = true)
 |-- AgencyName: string (nullable = true)
 |-- AgencyType: string (nullable = true)
 |-- City: string (nullable = true)
 |-- State: string (nullable = true)
 |-- Year: string (nullable = true)
 |-- Month: string (nullable = true)
 |-- Incident: string (nullable = true)
 |-- CrimeType: string (nullable = true)
 |-- CrimeSolved: string (nullable = true)
 |-- VictimSex: string (nullable = true)
 |-- VictimAge: string (nullable = true)
 |-- VictimRace: string (nullable = true)
 |-- VictimEthnicity: string (nullable = true)
 |-- PerpetratorSex: string (nullable = true)
 |-- PerpetratorAge: string (nullable = true)
 |-- PerpetratorRace: string (nullable = true)
 |-- PerpetratorEthnicity: string (nullable = true)
 |-- Relationship: string (nullable = true)
 |-- Weapon: string (nullable = true)
 |-- VictimCount: string (nullable = true)
 |-- PerpetratorCount: string (nullable = true)
 |-- RecordSource: strin

Row(RecordID='000001', AgencyCode='AK00101', AgencyName='Anchorage', AgencyType='Municipal Police', City='Anchorage', State='Alaska', Year='1980', Month='January', Incident='1', CrimeType='Murder or Manslaughter', CrimeSolved='Yes', VictimSex='Male', VictimAge='14', VictimRace='Native American/Alaska Native', VictimEthnicity='Unknown', PerpetratorSex='Male', PerpetratorAge='15', PerpetratorRace='Native American/Alaska Native', PerpetratorEthnicity='Unknown', Relationship='Acquaintance', Weapon='Blunt Object', VictimCount='0', PerpetratorCount='0', RecordSource='FBI')

In [5]:
# Count null values in each column
df.select([count(when(col(c).isNull(), c)).alias(c) for c in df.columns]).show()

+--------+----------+----------+----------+----+-----+----+-----+--------+---------+-----------+---------+---------+----------+---------------+--------------+--------------+---------------+--------------------+------------+------+-----------+----------------+------------+
|RecordID|AgencyCode|AgencyName|AgencyType|City|State|Year|Month|Incident|CrimeType|CrimeSolved|VictimSex|VictimAge|VictimRace|VictimEthnicity|PerpetratorSex|PerpetratorAge|PerpetratorRace|PerpetratorEthnicity|Relationship|Weapon|VictimCount|PerpetratorCount|RecordSource|
+--------+----------+----------+----------+----+-----+----+-----+--------+---------+-----------+---------+---------+----------+---------------+--------------+--------------+---------------+--------------------+------------+------+-----------+----------------+------------+
|       0|         0|         0|         0|   0|    0|   0|    0|       0|        0|          0|        0|        0|         0|              0|             0|             0|        

# Queries

### 1. Which are the 5 states with higher number of unsolved crimes

In [106]:
unsolvedCrimesPerState = df.where(df.CrimeSolved == 'No').groupBy("State").count().sort(F.desc("count")).limit(5)
unsolvedCrimesPerState.collect()
#unsolvedCrimesPerState.write.csv("./output/unsolvedCrimesPerState.csv", mode="overwrite", header=True)

[Row(State='California', count=36369),
 Row(State='New York', count=22612),
 Row(State='Texas', count=14680),
 Row(State='Florida', count=10647),
 Row(State='Illinois', count=10026)]

### 2. Which has been the most violent years of the 21st century

In [125]:
columnToNumber = udf(lambda x: int(x), IntegerType())
df = df.withColumn("Year", columnToNumber("Year"))
mostViolentYear21thCentury = df.where(df.Year >= 2000).groupBy("Year").count().withColumnRenamed("count", "Count").sort(F.desc("Count")).limit(5)
mostViolentYear21thCentury.collect()
#mostViolentYear21thCentury.write.csv("./output/mostViolentYear21thCentury.csv", mode="overwrite", header=True)

[Row(Year=2007, Count=17303),
 Row(Year=2006, Count=17275),
 Row(Year=2005, Count=16836),
 Row(Year=2003, Count=16512),
 Row(Year=2002, Count=16268)]

### 3. Number of crimes per gender

In [108]:
crimesPerGender = df.groupBy("VictimSex").count().withColumnRenamed("count", "Count").sort(F.desc("Count"))
crimesPerGender.collect()

[Row(VictimSex='Male', Count=494125),
 Row(VictimSex='Female', Count=143345),
 Row(VictimSex='Unknown', Count=984)]

### 4. Which was the most violent month of each year

In [116]:
yearMonth = df.select(df.Year, df.Month).groupBy(df.Year, df.Month).count().withColumnRenamed("count", "Count").sort(df.Year)

agg = yearMonth.groupBy(yearMonth.Year).agg(F.max(yearMonth.Count)).withColumnRenamed("max(Count)", "Crimes")

result = agg.join(yearMonth, [agg.Crimes == yearMonth.Count, agg.Year == yearMonth.Year], 'inner').select(agg.Year, yearMonth.Month, agg.Crimes).limit(5)

result.collect()

[Row(Year=1980, Month='August', Crimes=2371),
 Row(Year=1981, Month='January', Crimes=1987),
 Row(Year=1982, Month='August', Crimes=1832),
 Row(Year=1983, Month='December', Crimes=1913),
 Row(Year=1984, Month='December', Crimes=1699)]

### 5. Number of crimes against minors and percentage over total crimes per year

In [ ]:
df = df.withColumn("VictimAge", columnToNumber(df.VictimAge))

crimeMinors = df.where(df.VictimAge <= 18).groupBy(df.Year).agg(F.count(F.lit(1)).alias("CrimesMinors"))

totalCrimes = df.groupBy(df.Year).agg(F.count(F.lit(1)).alias("TotalCrimes"))

result = crimeMinors.join(totalCrimes, crimeMinors.Year == totalCrimes.Year, 'inner') \
    .select(crimeMinors.Year, crimeMinors.CrimesMinors, totalCrimes.TotalCrimes, \
    ((crimeMinors.CrimesMinors * 100) / totalCrimes.TotalCrimes).alias("PercentageOverTotal")) \
    .sort(F.desc("Year"))

result.collect()

### 6. State with most number of murders with explosives

In [155]:
stateExplosives = df.where(df.Weapon == "Explosives").groupBy(df.State).count().sort(F.desc("count")).limit(5)
stateExplosives.collect()
# Oklahoma may need more agressive explosive regulations xd

[Row(State='Oklahoma', count=158),
 Row(State='Michigan', count=52),
 Row(State='California', count=50),
 Row(State='New York', count=45),
 Row(State='Missouri', count=18)]

### 7. Which States are Best at Solving Murders?

In [159]:
stateSolvedCrimes = df.where(df.CrimeSolved == 'Yes').groupBy("State").count().sort(F.desc("count")).limit(5)
stateSolvedCrimes.collect()

[Row(State='California', count=63414),
 Row(State='Texas', count=47415),
 Row(State='New York', count=26656),
 Row(State='Florida', count=26517),
 Row(State='Michigan', count=19014)]

### 8. Does Victim Race Affect Whether a Murder is Solved? 🤔

In [162]:
unsolvedByRace = df.where(df.CrimeSolved == 'No').groupBy("victimRace").count().sort(F.desc("count")).limit(5)
unsolvedByRace.collect()

[Row(victimRace='Black', count=101125),
 Row(victimRace='White', count=82236),
 Row(victimRace='Unknown', count=3084),
 Row(victimRace='Asian/Pacific Islander', count=2917),
 Row(victimRace='Native American/Alaska Native', count=920)]

### 9. Can We Predict the Age of a Killer?

In [195]:
def ageRange(age):
    if(age > 0 and age <= 10):
        return "0-10"
    elif(age > 10 and age <= 20):
        return "11-20"
    elif(age > 20 and age <= 30):
        return "21-30"
    elif(age > 30 and age <= 40):
        return "31-40"
    elif(age > 40 and age <= 50):
        return "41-50"   
    elif(age > 50 and age <= 60):
        return "51-60"
    elif(age > 60 and age <= 70):
        return "61-70"
    elif(age > 70):
        return "71-99"
    else:
        return "Unknown"
           
df = df.withColumn("PerpetratorAge", columnToNumber(df.VictimAge))
ageRangeUDF = udf(ageRange)
result = df.withColumn("PerpetratorAgeRange", ageRangeUDF(df.PerpetratorAge)).groupBy("PerpetratorAgeRange").agg(count(F.lit(1)).alias("Crimes")).sort(col("Crimes").desc())
result.collect()

[Row(AgeRange='21-30', Crimes=208188),
 Row(AgeRange='31-40', Crimes=132473),
 Row(AgeRange='11-20', Crimes=100165),
 Row(AgeRange='41-50', Crimes=76991),
 Row(AgeRange='51-60', Crimes=41260),
 Row(AgeRange='71-99', Crimes=29891),
 Row(AgeRange='61-70', Crimes=22027),
 Row(AgeRange='0-10', Crimes=19015),
 Row(AgeRange='Unknown', Crimes=8444)]

### 10. What about the races of the most violent age range... 🤔

In [198]:
df.withColumn("PerpetratorAgeRange", ageRangeUDF(df.PerpetratorAge)).where(col("PerpetratorAgeRange") == "21-30").groupBy(df.PerpetratorRace).count().sort(F.desc("count")).collect()

[Row(PerpetratorRace='Black', count=74982),
 Row(PerpetratorRace='Unknown', count=69778),
 Row(PerpetratorRace='White', count=60619),
 Row(PerpetratorRace='Asian/Pacific Islander', count=1766),
 Row(PerpetratorRace='Native American/Alaska Native', count=1043)]